<a href="https://colab.research.google.com/github/JapiKredi/RAG_assignment_Research_papers/blob/main/RAG_assignment_Research_papers_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. <font color = red> Install and Import the Required Libraries

In [30]:
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding

In [3]:
# Import all the required Libraries
import openai
import pypdf
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import chromadb

In [4]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from sentence_transformers import CrossEncoder, util

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [5]:
!pip show langchain

Name: langchain
Version: 0.1.4
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## 2. <font color = red> Read, Process, and Chunk the PDF Files

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [7]:
# Set the API key
filepath = "/content/drive/My Drive/GenerativeAI/MateAI/"

with open(filepath + "Jasper_OpenAI_API_Key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [8]:
import os
os.environ["OPENAI_API_KEY"] = openai.api_key

## Load multiple and process documents

In [9]:
# Load and process the text files
loader = DirectoryLoader("/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/", glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [10]:
len(documents)

157

## Splitting, Chunking of text

In [11]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [12]:
len(texts)

711

In [13]:
texts[10]

Document(page_content='1024) since we break batches into multiple updates. See Table 1 in the appendix for exact numbers.\nNonoverlapping Inference To train on or evaluate a sequence longer than Ltokens, it is typical\nto segment the sequence into L-length subsequences and train on or evaluate them independently.\nUnless otherwise stated, we use nonoverlapping inference to report perplexity scores.\nExtrapolation During Inference Formally, the functions that deﬁne a transformer layer are ag-\nnostic to input length;3they map from some arbitrary, unﬁxed number of input vectors to the same\nnumber of output vectors. When transformers are applied to data that is inherently sequential, like\ntext, positional information is injected into the inputs in various ways.\nVaswani et al. (2017) discussed two options for embedding positions into vectors to be added to word\nembeddings: learning embeddings for speciﬁc positions and unlearned sinusoidal embeddings. They', metadata={'source': '/conten

## Embedding the text via regular HuggingFace process

In [14]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

### Creating Chroma Vector Database

In [15]:
# Define the path where chroma collections will be stored
chroma_data_path = '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

In [16]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = chroma_data_path

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [17]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [18]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## 4. <font color = red> Semantic Search

In this section, we will perform a semantic search of a query in the collections embeddings to get several top semantically similar results.

In [19]:
## Make a retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 1})

In [20]:
# Get a relevant query
query = "What is Flash attention?"

In [21]:
docs = retriever.get_relevant_documents("What is Flash attention?")

In [22]:
retriever.search_type

'similarity'

## Make a chain

In [23]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


## 6. Retrieval Augmented Generation

Now that we have the final top search results, we can pass it to an GPT 3.5 along with the user query and a well-engineered prompt, to generate a direct answer to the query along with citations, rather than returning whole pages/chunks.

In [24]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [25]:
# Querry 1: What is Flash attention?
query = "What is Flash attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Flash attention is a type of attention mechanism used in machine learning and natural language processing
tasks. It is designed to be more efficient and faster than other types of attention, such as exact,
approximate, and sparse attention, by reducing the number of memory accesses required. It can be up to 3 times
faster than the PyTorch implementation and is faster than other types of attention across all sequence
lengths.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Flash-attention.pdf


In [26]:
# Querry 2: What is self attention?
query = "What is self-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 I don't know.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/attention_all_you_need.pdf


In [27]:
# Querry 3: What is multi head attention?
query = "What is multi head attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 I don't know.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/attention_all_you_need.pdf


In [28]:
# Querry 4: What does IO-aware mean?
query = "What does IO-aware mean?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


I am unable to answer this question as the term "IO-aware" is not mentioned in any of the provided context.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/GPT_4_tech_report.pdf


In [29]:
# Querry 5: What is tiling in flash-attention?
query = "What is tiling in flash-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Tiling in FlashAttention refers to the technique of dividing the attention matrix into smaller blocks, which
is also used in the algorithm of Rabe and Staats [66]. This helps reduce the memory footprint and the amount
of memory accesses, which is important for optimizing runtime.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Flash-attention.pdf


In [30]:
# Querry 6: What tools can be used with toolformer?
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 I don't know.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Augmenting LLMs Survey.pdf


In [31]:
# Querry 7: What are the best retrieval augmentations for LLMs?
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Dense and sparse retrievers are both commonly used and effective retrieval augmentations for LLMs.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Augmenting LLMs Survey.pdf


## Deleting the Chroma Vector Database

In [ ]:
# Define the path where chroma collections will be stored
# chroma_data_path = '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

In [ ]:
# !zip -r db.zip ./db
!zip -r ChromaDB_Data.zip '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

In [ ]:
# To cleanup, you can delete the collection
#vectordb.delete_collection()
#vectordb.persist()

# delete the directory
#!rm -rf '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

## Starting again loading the db

restart the runtime

In [ ]:
#!unzip db.zip
#!unzip '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data.zip'

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [ ]:
# Set the API key
#filepath = "/content/drive/My Drive/GenerativeAI/MateAI/"

#with open(filepath + "Jasper_OpenAI_API_Key.txt", "r") as f:
#  openai.api_key = ' '.join(f.readlines())

In [ ]:
#import os
#os.environ["OPENAI_API_KEY"] = openai.api_key

# Solution 2


***Model design***
- Multiple Files - PDFs
- LangChain
- Chroma Vector database
- Multi-doc retriever
- Instuctor Embeddings (Hugging Face)
- Local LLM: Flan-T5-XL
- No Open AI




## 1. <font color = red> Install and Import the Required Libraries

In [ ]:
!pip -q install langchain tiktoken chromadb pypdf transformers InstructorEmbedding
!pip -q install accelerate bitsandbytes

In [ ]:
!pip show langchain

In [ ]:
# We check if the LLM is working properly by asking a simple question:
print(local_llm('What is the currency of India?'))

## Setting up LangChain


In [ ]:
import os

from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

## 2. <font color = red> Read, Process, and Chunk the PDF Files

## Load multiple and process documents

In [ ]:
# Load and process the text files
loader = DirectoryLoader("/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/", glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [ ]:
len(documents)

## Splitting, Chunking of text

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

In [ ]:
texts[2]

### Loading Instructor Embeddings

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl",
                                              load_in_8bit=True,
                                              device_map='auto',
                                            #   torch_dtype=torch.float16,
                                            #   low_cpu_mem_usage=True,

                                              )

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# Loading the Instructor Embeddings from Hugging Face
# Using GPU -> "cuda"

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

### Creating a Chroma Vector Database


In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)